In [ ]:
import os
import pandas as pd
import numpy as np

from glob import glob
from PIL import Image
import cv2

# import data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.models.widgets import Tabs

# import data augmentation
import albumentations as A
from ast import literal_eval

# Analysis

## Load data

In [ ]:
"""Image
"""
train_dir = '../input/global-wheat-detection/train/'
test_dir = '../input/global-wheat-detection/test/'

train_path = glob(train_dir + '/*')
test_path = glob(test_dir + '/*')

print('Number of train path: ', len(train_path))
print('Number of test path: ', len(test_path))

* We only have 10 test images and those are very low.
* we have 3422 train images which are quite small for training a deep neural network model so data augmentation and transfer learning techniques will sure come in handy a lot.

In [ ]:
"""Annotations
"""
train_df = pd.read_csv('../input/global-wheat-detection/train.csv')
print(train_df.head(5))
print('Number of rows are: ', train_df.shape[0])

+ Each row cosists of an image id, shape of image,Bbox and source details.

## Basic check

In [ ]:
"""Shape check
"""
print('Shapes of width: ', train_df['width'].unique().item())
print('Shapes of height: ', train_df['height'].unique().item())

+ All train images have shape 1024x1024

In [ ]:
"""Create a dataframe with splitted boxes.
"""
all_train_images = pd.DataFrame([i.split('/')[-1][:-4] for i in train_path])
all_train_images.columns = ['image_id']

# Merge with the rest columns
all_train_images = all_train_images.merge(train_df, on='image_id', how = 'left')

# Fill nan boxes with zeros 
all_train_images['bbox'] = all_train_images.bbox.fillna('[0,0,0,0]')

# Split bbox infos to 4 cols
# Create the sub-dataframe which has splitted box infos
bbox_items = all_train_images.bbox.str.split(',', expand = True)

all_train_images['x_min'] = bbox_items[0].str.strip('[ ').astype(float)
all_train_images['y_min'] = bbox_items[1].str.strip(' ').astype(float)
all_train_images['width'] = bbox_items[2].str.strip(' ').astype(float)
all_train_images['height'] = bbox_items[3].str.strip('] ').astype(float)

In [ ]:
"""Check images with no bboxes.
"""
# Removes img w/no bboxes in all_train_images dataframe
all_train_images = all_train_images[all_train_images.width!=None]

# Compare processed dataframe to init dataframe
print('There are {} images with no bounding box!'.format(len(all_train_images)- len(train_df)))

## Visualize imgs

In [ ]:
# This code is used for connecting boxes with id. 
def get_all_bboxes(df, image_id):
    image_bboxes = df[df.image_id == image_id]   
    bboxes = []
    for _,row in image_bboxes.iterrows():
        bboxes.append((row.x_min, row.y_min, row.width, row.height))
        
    return bboxes

def plot_image_examples(df, rows=3, cols=3):
    fig, axs = plt.subplots(rows, cols, figsize=(25,20))
    for row in range(rows):
        for col in range(cols):
            idx = np.random.randint(len(df), size=1)[0]
            img_id = df.iloc[idx].image_id
       
            img = Image.open(train_dir + img_id + '.jpg')
            axs[row, col].imshow(img)
            
            bboxes = get_all_bboxes(df, img_id)
            
            for bbox in bboxes:
                rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
                axs[row, col].add_patch(rect)
            
            axs[row, col].axis('off')
            
            
# Sample images with bounding box.
plot_image_examples(all_train_images)

## Bboxes analysis

In [ ]:
def hist_hover(dataframe, column, colors=["#94c8d8", "#ea5e51"], bins=30, title=''):
    """Count the appearance of values in one column and plot the histogram.
    """
    hist, edges = np.histogram(dataframe[column], bins = bins)
    
    hist_df = pd.DataFrame({column: hist,
                             "left": edges[:-1],
                             "right": edges[1:]})
    hist_df["interval"] = ["%d to %d" % (left, right) for left, 
                           right in zip(hist_df["left"], hist_df["right"])]

    src = ColumnDataSource(hist_df)
    plot = figure(plot_height = 400, plot_width = 600,
          title = title,
          x_axis_label = column,
          y_axis_label = "Count")    
    plot.quad(bottom = 0, top = column,left = "left", 
        right = "right", source = src, fill_color = colors[0], 
        line_color = "#35838d", fill_alpha = 0.7,
        hover_fill_alpha = 0.7, hover_fill_color = colors[1])
        
    hover = HoverTool(tooltips = [('Interval', '@interval'),
                              ('Count', str("@" + column))])
    plot.add_tools(hover)
    
    output_notebook()
    show(plot)

In [ ]:
def get_area_bbox(bbox):
    bbox = literal_eval(bbox)
    return bbox[2]*bbox[3]

all_train_images['area_bboxes'] = all_train_images['bbox'].apply(get_area_bbox)

In [ ]:
hist_hover(all_train_images, 'area_bboxes', title='Bboxes area distribution')

In [ ]:
print('Minimum area of bboxes: ', all_train_images.area_bboxes.min())

In [ ]:
print('Minimum area of bboxes (>0): ', all_train_images[all_train_images['area_bboxes'] > 0].min()['area_bboxes'])

As organizers say, there are many bounding boxes for each image, and not all images include wheat heads / bounding boxes.

### Bboxes analysis by area

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)

"""Histogram of number of bboxes per image.
"""
fig.set_size_inches(16,5)
ax1.hist(all_train_images['image_id'].value_counts(), bins=30)
ax1.set_title('Number of spikes per image')
ax1.set_xlabel('Bounding boxes')
ax1.set_ylabel('Images')


ax2.set_xlim(100, 125)
ax2.set_ylim(0,10)
ax2.hist(all_train_images['image_id'].value_counts(), bins=30)
ax2.set_title('Zooming left part of histogram')
ax2.set_xlabel('Bounding boxes')
ax2.set_ylabel('Images')
ax2.grid(True)

+ Max number of bounding boxes is 116, whereas min (annotated) number is 1.
+ Most of the images have 20-50 wheat heads.

In [ ]:
# Calculate total bboxes area per img
area_per_image = all_train_images.groupby(by='image_id').sum().reset_index()

"""area_per_image_precent: for each image, calculate bboxes area/image area.
"""
area_per_image_precent = area_per_image.copy()
area_per_image_precent['area_bboxes'] = area_per_image_precent['area_bboxes']/(1024*1024)

hist_hover(area_per_image_precent, 'area_bboxes', title='Percentage of image area covered by bounding boxes')

+ This looks like a nice normal distribution! 20-40% of the image area is covered by the bounding boxes.
+ This observation can be used to validate the predictions of the resulting model. The percentage of predicted bounding boxes area should be normally distributed too.
+ We can also see that the maximum is actually greater than 100%. This means that the bounding boxes are overlapping.

In [ ]:
"""Image with small covered bboxes.
"""
small_boxes_ids = all_train_images[(all_train_images['area_bboxes'] < 50) & (all_train_images['area_bboxes'] > 0)].image_id

plot_image_examples(all_train_images[all_train_images.image_id.isin(small_boxes_ids)])

* If you look very close, you can probably see those tinyest bounding boxes near the corners and borders of the images. Probably, the boundries were drawn first, than the images were cut into several ones. That is why we see those strange small bounsing boxes in the corners.  
* It is not necessary to clean these, because they won't have much effect on the IOU metric.

In [ ]:
large_boxes_ids = all_train_images[all_train_images['area_bboxes'] > 200000].image_id

plot_image_examples(all_train_images[all_train_images.image_id.isin(large_boxes_ids)])

In [ ]:
small_area_percent = area_per_image_precent[area_per_image_precent['area_bboxes'] < 7].image_id

plot_image_examples(all_train_images[all_train_images.image_id.isin(small_area_percent)])

### Bboxes analysis per source

In [ ]:
avg_area = all_train_images.groupby('source', as_index = False)['area_bboxes'].mean()
avg_area

In [ ]:
plt.figure(figsize=(12,8))
plt.bar(avg_area['source'],avg_area['area_bboxes'],align='center')
plt.title('Bar plot for avg Bbox area feature')
plt.xlabel('Source')
plt.ylabel('Avg Area')
plt.show()

In [ ]:
counts = dict(all_train_images.source.value_counts())
fig,ax = plt.subplots(figsize=(8,8))

wedges, texts, autotexts = ax.pie(list(counts.values()), autopct='%1.1f%%',shadow=True, startangle=90);
ax.legend(wedges, list(counts.keys()),
          title="Sources",
          loc="center",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.setp(autotexts, size=15);

ax.set_title("Data Distribution based on Sources")
plt.show()
sns.countplot(x='source', data=all_train_images)

+ Most of the images are produced by arvalis_1 source followed by ethz_1.
+ I noticed that images came from different sources and each source contributed different amout of images for the training.
+ Total 7 sources out of which arvalis_2 and inrae_1 are minority.

In [ ]:
all_train_images['source'] = all_train_images['source'].fillna('No source')
all_train_images.source.unique()

print('No of images with no bounding boxes are',all_train_images[all_train_images['source']=='No source'].shape[0])

In [ ]:
"""Images w/o source is image w/o bboxes.
"""
plot_image_examples(all_train_images[all_train_images.source == 'No source'])

In [ ]:
plot_image_examples(all_train_images[all_train_images['source'] == 'ethz_1'])

+ Density of wheat heads are more in this source of images.

In [ ]:
plot_image_examples(all_train_images[all_train_images['source'] == 'arvalis_3'])

+ Images have very low bounding boxes to high bounding boxes count.
+ Some images have a lot of background data compared to wheat heads.
+ Images might be taken in different light conditions.

In [ ]:
plot_image_examples(all_train_images[all_train_images['source'] == 'usask_1'])

+ Images have very low bounding boxes to high bounding boxes count.
+ Some images have a lot of background data compared to wheat heads.
+ Images might be taken in different light conditions.
+ Images contain mostly leaves instead of wheat heads.

In [ ]:
plot_image_examples(all_train_images[all_train_images.source == 'rres_1'])

* Color of the leaves showcase yellow tint rather than green.
* Avergae number of bounding boxes.
* Mostly taken in dark lighting conditions.

In [ ]:
plot_image_examples(all_train_images[all_train_images.source == 'arvalis_1'])

* Images look so bright.
* Most of the images are taken in bright day light conditions.
* Images have average to more bounding boxes, so its good.
* Most images have green and brown tint.

In [ ]:
plot_image_examples(all_train_images[all_train_images.source == 'arvalis_2'])

* Images are taken in brighter conditions.
* Less wheat heads compared to other stuff in the images.
* Wheat heads are almost same color as leaves...so it might be harder to detect a litle bit.

## Images analysis

In [ ]:
images_df = pd.DataFrame(all_train_images.image_id.unique())
images_df.columns = ['image_id']

### Brightness

In [ ]:
def get_image_brightness(img):
    img = cv2.imread(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return np.array(img).mean()

def get_brightness(df):
    brightness = [] 
    for _, row in df.iterrows():
        img_id = row.image_id  
        image_name = train_dir + img_id + '.jpg'
        brightness.append(get_image_brightness(image_name))
        
    brightness_df = pd.DataFrame(brightness)
    brightness_df.columns = ['brightness']
    df = pd.concat([df, brightness_df], ignore_index = True, axis = 1)
    df.columns = ['image_id', 'brightness']
    return df

In [ ]:
brightness_df = get_brightness(images_df)
all_train_images = all_train_images.merge(brightness_df, on = 'image_id')

all_train_images

In [ ]:
hist_hover(all_train_images, 'brightness', title='Images brightness distribution')

In [ ]:
bright_ids = all_train_images[all_train_images['brightness'] > 130]
print('Number of bright images (greater than 170): {}'.format(len(bright_ids.image_id)), end='\n-------\n')
print('Source statistics of images: \n', bright_ids.source.value_counts(), end='\n-------\n')

In [ ]:
"""Plot dark imgs.
"""
dark_ids = all_train_images[all_train_images['brightness'] < 30].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(dark_ids)])

On some of the images, it is even hard for human to see the spikes!

In [ ]:
"""Plot light imgs.
"""
bright_ids = all_train_images[all_train_images['brightness'] > 140].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(bright_ids)])

These are very different from the dark ones. Some filters needed here to make the spikes more clear.

### Green and yellow images

I would like to plot images with different dominant colors. The idea is that the most green images will represent healthy plants. The most yellow images will contain plants close to maturity. The most brown images will have ground on them.

In [ ]:
def get_percent_color_pix(image, mode):
    """Calculate percent of specific color in image.
    """
    # convert to hsv
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros((image.shape[0],image.shape[1], 3), np.uint8)
    
    if mode == 'green': 
        # get the green mask
        hsv_lower = (40, 40, 40) 
        hsv_higher = (70, 255, 255)
        mask = cv2.inRange(hsv, hsv_lower, hsv_higher)
        
    elif mode == 'yellow': 
        # get the green mask
        hsv_lower = (25, 40, 40) 
        hsv_higher = (35, 255, 255)
        mask = cv2.inRange(hsv, hsv_lower, hsv_higher)  
        
    return (np.sum(mask)) / 255 / (1024 * 1024)

def add_color_pixels_percentage(df, mode):
    """Add above result to df. 
    """
    color = []
    for _, row in df.iterrows():
        img_id = row.image_id  
        image = cv2.imread(train_dir + img_id + '.jpg')
        color.append(get_percent_color_pix(image, mode))
        
    color_df = pd.DataFrame(color)
    
    if mode == 'green':
        color_df.columns = ['green_pixels']
        df = pd.concat([df, color_df], ignore_index=True, axis=1)
        df.columns = ['image_id', 'green_pixels']
        
    elif mode == 'yellow':
        color_df.columns = ['yellow_pixels']
        df = pd.concat([df, color_df], ignore_index=True, axis=1)
        df.columns = ['image_id', 'yellow_pixels']
    
    return df

In [ ]:
# add a column with the percentage of green and yellow pixels
green_pixels_df = add_color_pixels_percentage(images_df, mode ='green')
yellow_pixels_df = add_color_pixels_percentage(images_df, mode ='yellow')

all_train_images = all_train_images.merge(green_pixels_df, on='image_id')
all_train_images = all_train_images.merge(yellow_pixels_df, on='image_id')

In [ ]:
"""Green pixels histogram.
"""
hist_hover(all_train_images, 'green_pixels', title='Percentage of green pixels distribution', colors=['#c3ea84', '#3e7a17'])

+ Surprisingly, the most green images have only aoung 60% green pixels.

+ The majority of the images are not green at all! Most probably, they are more yellow and are the images of the plants close to harvest.

In [ ]:
green_ids = all_train_images[all_train_images['green_pixels'] > 0.6].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(green_ids)])

The most green images mostly contain the plants with very small spikes, which are just starting to appear.

In [ ]:
not_green_ids = all_train_images[all_train_images['green_pixels'] < 0.03].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(not_green_ids)])

In [ ]:
"""Yellow pixels histogram.
"""
hist_hover(all_train_images, 'yellow_pixels', title='Percentage of yellow pixels distribution', colors=['#fffedb', '#fffeab'])

In [ ]:
yellow_ids = all_train_images[all_train_images['yellow_pixels'] > 0.6].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(yellow_ids)])

### Observations from the above analysis
* It is evident from the above image analysis that each type of source has specific types of wheat images. This is probably dependent on the region of data collection and time of the year when the particular data was collected.
* There are some images with no bounding boxes at all.
* Count of bounding boxes will vary from a min of 1 to max of 116.
* Most of the images conatin an average of 20 to 60 bounding boxes.
* Images given are taken from different lighting conditions at differen plant growth level.
* Bounding boxes are messy!
* Giant bounding boxes should be filtered out by area and removed before model training.
* Micro bounding boxes. These can stay. They won't have much effect on the IOU metric.
* Some spikes are not surrounded by a bounding box (missing bounding boxes).

# Data augmentation
Data augmentation is critical in this competition, because there is a relatively small training set. Data augmentation will allow to build robust models under given circumstances. What augmentations/filers might work:

+ flipping images horizontally and vertically, because the orientation of original images is different;
+ crop-resize, because we can see spikes at different zoom levels;
+ different filters to adjust the lighting conditions. I suggest looking at this competition for example.
What to do with caution:

**rotation might not work, because rotation messes up the bounding boxes**

In [ ]:
# setup an example augmentation pipeline
# be sure to use bbox safe functions for data augmentation
transforms = A.Compose([
    A.RandomSizedBBoxSafeCrop(512, 512, erosion_rate=0.0, interpolation=1, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.OneOf([A.RandomContrast(),
                A.RandomGamma(),
                A.RandomBrightness()], p=1.0),
    A.CLAHE(p=1.0)], p=1.0, bbox_params=A.BboxParams(format='coco', label_fields=['category_id']))

In [ ]:
def apply_transform(transforms, df, n_transforms=3):
    """Apply augmented transformation in random image.
    """
    # Get random image by index
    idx = np.random.randint(len(df), size=1)[0]
    image_id = df.iloc[idx].image_id
    
    # Get bboxes info
    bboxes = []
    for _, row in df[df.image_id == image_id].iterrows():
        bboxes.append([row.x_min, row.y_min, row.width, row.height])
        
    image_path = train_dir + image_id + '.jpg'
    image = cv2.imread(image_path)
    
    fig, axs = plt.subplots(1, n_transforms+1, figsize=(15,7))
    
    # plot the original image
    axs[0].imshow(image)
    axs[0].set_title('original')
    for bbox in bboxes:
        rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
        axs[0].add_patch(rect)
        
    # apply transforms n_transforms times
    for i in range(n_transforms):
        params = {'image': np.asarray(image),
                  'bboxes': bboxes,
                  'category_id': [1 for j in range(len(bboxes))]}
        
        # Get transformation
        augmented_boxes = transforms(**params)
        
        bboxes_aug = augmented_boxes['bboxes']
        image_aug = augmented_boxes['image']
        
        # plot the augmented image and augmented bounding boxes
        axs[i+1].imshow(image_aug)
        axs[i+1].set_title('augmented_' + str(i+1))
        for bbox in bboxes_aug:
            rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
            axs[i+1].add_patch(rect)
            
    plt.show()

In [ ]:
apply_transform(transforms, all_train_images, n_transforms=3)

In [ ]:
apply_transform(transforms, all_train_images, n_transforms=3)

In [ ]:
apply_transform(transforms, all_train_images, n_transforms=3)